In [1]:
#default_exp ghtop

# ghtop

> I cannot believe this library name is not already taken on pypi.

In [4]:
#export
import time, sys, signal, shutil, os, json, enlighten, emoji, blessed
from ghtop.dashing import *
from collections import defaultdict

from fastcore.utils import *
from ghapi.all import *

In [5]:
term = Terminal()
logfile = Path("log.txt")

In [58]:
def github_auth_device():
    auth = GhDeviceAuth()
    wb = input(f"""First copy your one-time code: {term.yellow}{auth.user_code}{term.normal}
    Then visit {auth.verification_uri} in your browser, and paste the code when prompted.
    Shall we try to open the link for you? [y/n] """)
    if wb[0].lower()=='y': auth.open_browser()

    print ("Waiting for authorization...", end='')
    token = auth.wait(lambda: print('.', end=''))
    print("Authenticated with GitHub!")
    return token

In [59]:
#export
def exit(msg):
    print(msg, file=sys.stderr)
    sys.exit()

In [60]:
#export
def get_token():
    path = Path.home()/".ghtop_token"
    if path.is_file():
        try: return path.read_text().strip()
        except: exit("Error reading token")

    token = github_auth_device()
    path.write_text(token)
    return token

In [61]:
token = get_token()

In [63]:
def limit_cb(rem,quota):
    if remaining_apis >= 1000: return
    print("WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING ")
    print("Remaining calls: " + str(remaining_apis))
    print("WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING ")

In [64]:
api = GhApi(limit_cb=limit_cb)

In [65]:
#export
def fetch_events(): return api.activity.list_public_events(per_page=100)

In [66]:
#export
def read_json_log():
    try: return logfile.read_json()
    except FileNotFoundError: return []

In [6]:
EVENT_EMOJI_MAPPING = defaultdict(str, dict(
    CreateEvent=':new:',
    DeleteEvent=':x:',
    DeploymentEvent=':rocket:',
    ForkEvent=':fork_and_knife:',
    GistEvent=':notepad:',
    IssueCommentEvent=':speech_bubble:',
    LabelEvent=':label:',
    PushEvent=':fist:',
    ReleaseEvent=':rocket:',
    StarEvent=':star:',
    WatchEvent=':eyes:'
))

In [67]:
printed_event_ids = {}

In [1]:
def emo(s): return emoji.emojize(s)
def show(s): print(emo(s))

def _title(d): return f'("{d.title[:50]}...")'
def _repo(r,i=22): return f'on repo {r[:i]}'

def _issue(e, login, repo, detl, emoji):
    issue = e.payload.issue
    return f':{emoji}: {login} {detl} {issue # {issue.number}} {_repo(r)} {_title(issue)}'

def _issue_comment(e, login, repo): return _issue(e, login, repo, 'commented on', 'speech_balloon')
def _issue_opened(e, login, repo): return _issue(e, login, repo, 'opened', 'closed_mailbox_with_raised_flag')
def _issue_closed(e, login, repo): return _issue(e, login, repo, 'closed', 'star')
def _release(e, login, repo): return f':rocket: {login} released {e.payload.release.tag_name} of {repo}'

In [70]:
#export
def print_event(e, commits_counter):
    if e.id in printed_event_ids: return
    printed_event_ids[e.id] = 1
    login,repo = e.actor.login,e.repo.name
    # Don't print bot activity (there is a lot!)
    if "bot" in login or "b0t" in login: return

    if e.type == "ReleaseEvent": show(term.firebrick3(_release(e, login, repo)))
    elif e.type in ("PublicEvent","ForkEvent","CreateEvent","MemberEvent"): return
    elif e.type == "IssuesEvent":
        if e.payload.action == 'closed': show(_issue_closed(e, login, repo))
        elif e.payload.action == 'opened': show(_issue_opened(e, login, repo))
    elif e.type == "IssueCommentEvent": show(term.white(_issue_comment(e, login, repo)))
    elif e.type == "PushEvent":
        for c in e.payload.commits: commits_counter.update()
    elif e.type == "PullRequestEvent":
        pr_emoji,pr_color = ("white_heavy_check_mark",term.green) if e.payload.action == "closed" else ("sparkles",term.yellow)
        show(pr_color(f':{pr_emoji}: {login} {e.payload.action} a pull request {_repo(repo,20)} {_title(e.payload.pull_request)}'))
    elif e.type == "SecurityAdvisoryEvent": return print(term.blink("SECURITY ADVISORY"))

In [71]:
ev = [o for o in fetch_events() if o.type=='PullRequestEvent']
[(i,o.type) for i,o in enumerate(ev)]
printed_event_ids={}
for e in ev: print_event(e,None)

✨ GregJMitchell opened a pull request on repo Oxalisviolacea/viewi ("Javascript...")
✅ jdrucza closed a pull request on repo jdrucza/bet-refinery ("Create Blog “nba-championship-odds-2020”...")
✨ uxkjaer opened a pull request on repo uxkjaer/ui5-ecosyste ("pukll...")
✅ bt-admin closed a pull request on repo brain-tec/account_eb ("Syncing from upstream Noviat/account_ebics (12-gen...")
✨ tdeenes opened a pull request on repo Ikuyadeu/vscode-R ("Find in topic (help panel) #463...")


In [72]:
#export    
def write_logs(events):
    Path("tmp.log").write_text(json.dumps(events, indent=2))
    shutil.move("tmp.log", logfile)

def tail_events():
    manager = enlighten.get_manager()
    commits = manager.counter(desc='Commits', unit='commits', color='green')
    while True:
        events = fetch_events()
        log = read_json_log()
        combined = sorted(log + events, key=lambda x: int(x["id"]))
        write_logs(combined)
        for x in combined: print_event(x, commits)
        time.sleep(0.2)

In [73]:
combined = (fetch_events()+read_json_log()).sorted(using_attr(int,"id"))

In [76]:
#export
def watch_users():
    users,users_events = defaultdict(int),defaultdict(lambda: defaultdict(int))
    while True:
        for x in fetch_events():
            login = e.actor.login
            users[login] += 1
            users_events[login][x.type] += 1

        print (term.clear())
        print ("User".ljust(30), "Events".ljust(6), "PRs".ljust(5), "Issues".ljust(6), "Pushes".ljust(7))
        sorted_users = sorted(users.items(), key = lambda kv: (kv[1], kv[0]), reverse=True)
        for i in range(20):
            u = sorted_users[i]
            ue = users_events[u[0]]
            print(u[0].ljust(30), str(u[1]).ljust(6), 
                  str(ue.get('PullRequestEvent', '')).ljust(5), 
                  str(ue.get('IssuesEvent', '')).ljust(6), 
                  str(ue.get('PushEvent', '')).ljust(7))
        time.sleep(1)

In [77]:
#export
def push_to_log(e):
    return f"{e.actor.login} pushed {len(e.payload.commits)} commits to repo {e.repo.name}"

In [78]:
def issue_to_log(e):
    login,repo = e.actor.login,e.repo.name
    if e.type == "IssuesEvent":
        action,issue = e.payload.action,e.payload.issue
        if action == 'closed': return emo(_issue_closed(e, login, repo))
        elif action == 'opened': return emo(_issue_opened(e, login, repo))
    elif e.type == "IssueCommentEvent": return emo(_issue_comment(e, login, repo))

In [79]:
def pr_to_log(e):
    login,repo = e.actor.login,e.repo.name
    emo("white_heavy_check_mark" if e.payload.action == "closed" else "sparkles")
    return f':{pr_emoji}: {login} {e.payload.action} a pull request {_repo(repo,20)} {_title(e.payload.pull_request)}'

In [80]:
#export
def release_to_log(e):
    login,repo = e.actor.login,e.repo.name
    return emo(_release(e, login, repo))

In [81]:
#export
def str_clean(s): return s[:95]

In [ ]:
def signal_handler(sig, frame):
    if sig != signal.SIGINT: return
    print(term.exit_fullscreen())
    print(term.clear())
    print(term.normal)
    sys.exit(0)

In [ ]:
#export
def quad_logs():
    term.enter_fullscreen()
    ui = HSplit(
            VSplit(
                Log(title='Issues', border_color = 2, color=7),
                Log(title='Commits', border_color = 2, color=3)
            ),
            VSplit(
                Log(title='Pull Requests', border_color = 2, color=4),
                Log(title='Releases', border_color = 2, color=5)
            ),
        )

    issues,commits = ui.items[0].items
    prs,releases = ui.items[1].items
    for o in issues,commits,prs,releases: o.append(" ")

    while True:
        for x in fetch_events():
            t = x["type"]
            if t == 'PushEvent': commits.append(str_clean(push_to_log(x)))
            elif t == 'IssuesEvent' or t == 'IssueCommentEvent': issues.append(str_clean(issue_to_log(x)))
            elif t == 'PullRequestEvent': prs.append(str_clean(pr_to_log(x)))
            elif t == 'ReleaseEvent': releases.append(str_clean(release_to_log(x)))
        ui.display()
        time.sleep(0.1)

In [ ]:
#export
def simple():
    while True:
        for x in fetch_events(): print(f"{x.actor.login} {x.type} {x.repo.name}")

In [ ]:
if len(sys.argv) < 2: exit("Usage: ghtop <tail|quad|users|simple>")
signal.signal(signal.SIGINT, signal_handler)

if sys.argv[1] == 'tail': tail_events()
elif sys.argv[1] == 'quad': quad_logs()
elif sys.argv[1] == 'users': watch_users()
elif sys.argv[1] == 'simple': simple()